In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import os


# 1 - FUNCTION TO EVALUATE MODEL

In [36]:
def evaluate_model(input_dir, feature_set_name, features, output_csv=None):
    # Number of folds
    n_folds = 5

    # Initialize lists to store all predictions and true values across all folds
    all_y_true = []
    all_y_pred = []

    # Initialize list to store fold-wise metrics
    fold_metrics = []

    # Loop through each fold
    for fold in range(1, n_folds + 1):
        # Load training and test set
        train = pd.read_csv(os.path.join(input_dir, f'training_data_fold{fold}.csv'))
        test = pd.read_csv(os.path.join(input_dir, f'test_data_fold{fold}.csv'))
        
        # Convert to log years
        y_train = np.log1p(train['TSI_days'] / 365)
        y_test = np.log1p(test['TSI_days'] / 365)

        # Specified feature set
        X_train = train[features]
        X_test = test[features]
        
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train, y_train)
        
        # Predict on the test data
        y_pred = model.predict(X_test)
        
        # Store the true values and predictions
        all_y_true.extend(y_test)
        all_y_pred.extend(y_pred)
        
        # Calculate and store metrics for the current fold
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"Fold {fold}: MSE = {mse}, MAE = {mae}, R² = {r2}")
        
        fold_metrics.append({
            'Feature_Set': feature_set_name, 
            'Fold': fold,
            'MSE': mse,
            'MAE': mae,
            'R²': r2
        })

    # Convert lists to numpy arrays
    all_y_true = np.array(all_y_true)
    all_y_pred = np.array(all_y_pred)

    # Calculate overall performance metrics across all folds
    overall_mse = mean_squared_error(all_y_true, all_y_pred)
    overall_mae = mean_absolute_error(all_y_true, all_y_pred)
    overall_r2 = r2_score(all_y_true, all_y_pred)

    # Print the overall performance metrics
    print("\nOverall Performance across all folds:")

    print(f"Overall MSE: {overall_mse}")
    print(f"Overall MAE: {overall_mae}")
    print(f"Overall R²: {overall_r2}")
    
    # Add overall metrics to the DataFrame
    fold_metrics.append({
        'Feature_Set': feature_set_name, 
        'Fold': 'Overall',
        'MSE': overall_mse,
        'MAE': overall_mae,
        'R²': overall_r2
    })

    metrics_df = pd.DataFrame(fold_metrics)
    # Save to CSV if output_csv is provided
    if output_csv:
        metrics_df.to_csv(output_csv, index=False)
        print(f"Performance metrics saved to {output_csv}")

    return metrics_df


# IMPACT OF BSPLINE SMOOTHING

In [37]:
features_original = ['genome_lrtt', 'genome_maf12c', 'genome_maf3c', 'genome_tips', 
                     'gag_lrtt', 'gag_maf12c', 'gag_maf3c', 'gag_tips',
                     'pol_lrtt', 'pol_maf12c', 'pol_maf3c', 'pol_tips',
                     'gp120_lrtt', 'gp120_maf12c', 'gp120_maf3c', 'gp120_tips', 
                     'gp41_lrtt', 'gp41_maf12c', 'gp41_maf3c', 'gp41_tips']

# Feature set with smoothed features (excluding meta-features)
features_smoothed_no_meta = ['genome_lrtt', 'genome_maf12c', 'genome_maf3c', 'genome_tips', 
                                'gag_lrtt', 'gag_maf12c', 'gag_maf3c', 'gag_tips', 
                                'pol_lrtt', 'pol_maf12c', 'pol_maf3c', 'pol_tips', 
                                'gp120_lrtt', 'gp120_maf12c', 'gp120_maf3c', 'gp120_tips',
                                'gp41_lrtt', 'gp41_maf12c', 'gp41_maf3c', 'gp41_tips']

# Feature set with smoothed features (including meta-features)
features_smoothed_with_meta = ['genome_lrtt', 'genome_maf12c', 'genome_maf3c', 'genome_tips', 
                                'gag_lrtt', 'gag_maf12c', 'gag_maf3c', 'gag_tips', 
                                'pol_lrtt', 'pol_maf12c', 'pol_maf3c', 'pol_tips', 
                                'gp120_lrtt', 'gp120_maf12c', 'gp120_maf3c', 'gp120_tips', 
                                'gp41_lrtt', 'gp41_maf12c', 'gp41_maf3c', 'gp41_tips', 
                                'lrtt_coeff_1', 'lrtt_coeff_3', 'lrtt_coeff_11']  # meta-features included

In [38]:
feature_sets = [
    # Unsmoothed Feature Set
    {'name': 'og_features', 'features': features_original},
    
    # B-Spline Feature Sets
    {'name': 'bspline_smoothed_features', 'features': features_smoothed_no_meta},
    {'name': 'bspline_smoothed_features_meta', 'features': features_smoothed_with_meta},
    
    # P-Spline Feature Sets (assuming similar structure as B-spline)
    {'name': 'pspline_smoothed_features', 'features': features_smoothed_no_meta},
    {'name': 'pspline_smoothed_features_meta', 'features': features_smoothed_with_meta},
]

# Directories for smoothed and unsmoothed data
datasets = {
    'B-spline': './data/derived/smoothed/bspline/',
    'P-spline': './data/derived/smoothed/pspline/',
    'Unsmoothed': './data/derived/unsmoothed/'
}

all_results = pd.DataFrame()

for feature_set in feature_sets:
    for dataset_type, input_dir in datasets.items():
        # Check if the feature set corresponds to the right dataset
        if ('bspline' in feature_set['name'] and dataset_type != 'B-spline') or \
           ('pspline' in feature_set['name'] and dataset_type != 'P-spline') or \
           ('og_features' in feature_set['name'] and dataset_type != 'Unsmoothed'):
            continue
        
        # Define the output CSV path
        output_csv = f'performance_metrics_{feature_set["name"]}_{dataset_type}.csv'
        
        # Debugging print statements
        print(f"Processing feature set: {feature_set['name']} with dataset: {dataset_type}")
        
        # Evaluate model
        metrics_df = evaluate_model(input_dir, feature_set['name'], feature_set['features'], output_csv)
        metrics_df['Dataset_Type'] = dataset_type #track dataset used
        all_results = pd.concat([all_results, metrics_df], ignore_index=True) #store results 


# Save all results to a single CSV file
all_results.to_csv('all_performance_metrics.csv', index=False)
print("All performance metrics saved to 'all_performance_metrics.csv'")

Processing feature set: og_features with dataset: Unsmoothed
Fold 1: MSE = 0.2038062486884934, MAE = 0.34326948757565195, R² = 0.2817053016705564
Fold 2: MSE = 0.1725005963977685, MAE = 0.2869412208977178, R² = 0.3375189641124452
Fold 3: MSE = 0.2492940591133556, MAE = 0.3791275696736444, R² = 0.44387923639879145
Fold 4: MSE = 0.24617496589511367, MAE = 0.39800094309986195, R² = 0.4239186328046335
Fold 5: MSE = 0.2537773935054384, MAE = 0.3768731519132005, R² = 0.44948972779028185

Overall Performance across all folds:
Overall MSE: 0.2237161060618955
Overall MAE: 0.3553385120056173
Overall R²: 0.532301873194166
Performance metrics saved to performance_metrics_og_features_Unsmoothed.csv
Processing feature set: bspline_smoothed_features with dataset: B-spline
Fold 1: MSE = 0.18985263595420712, MAE = 0.3420130415832152, R² = 0.33088341134127297
Fold 2: MSE = 0.1688087508225693, MAE = 0.28537364169117696, R² = 0.351697336431548
Fold 3: MSE = 0.24965268520335332, MAE = 0.37528296757763235, 

In [39]:
all_results

,Feature_Set,Fold,MSE,MAE,R²,Dataset_Type
0,og_features,1,0.203806,0.343269,0.281705,Unsmoothed
1,og_features,2,0.172501,0.286941,0.337519,Unsmoothed
2,og_features,3,0.249294,0.379128,0.443879,Unsmoothed
3,og_features,4,0.246175,0.398001,0.423919,Unsmoothed
4,og_features,5,0.253777,0.376873,0.449490,Unsmoothed
5,og_features,Overall,0.223716,0.355339,0.532302,Unsmoothed
6,bspline_smoothed_features,1,0.189853,0.342013,0.330883,B-spline
7,bspline_smoothed_features,2,0.168809,0.285374,0.351697,B-spline
8,bspline_smoothed_features,3,0.249653,0.375283,0.443079,B-spline
9,bspline_smoothed_features,4,0.233595,0.382661,0.453358,B-spline
